In [23]:
import os

In [24]:
fileList = os.listdir('.')
dbList = []
for file in fileList:
    if ('.db' in file):
        dbList.append(file)
dbList

['EXOPLANETS.db', 'test.db', 'user.db']

In [31]:
freshness_query = """WITH UPDATES AS(
  SELECT
    DATE_ADDED,
    COUNT(*) AS ROWS_ADDED
  FROM
    EXOPLANETS
  GROUP BY
    DATE_ADDED
),

NUM_DAYS_UPDATES AS (
  SELECT
    DATE_ADDED,
    JULIANDAY(DATE_ADDED) - JULIANDAY(LAG(DATE_ADDED)
      OVER(
        ORDER BY DATE_ADDED
      )
    ) AS DAYS_SINCE_LAST_UPDATE
  FROM
    UPDATES
)

SELECT
  *
FROM
  NUM_DAYS_UPDATES
WHERE
  DAYS_SINCE_LAST_UPDATE > 3;"""

In [32]:
totalquery = '.\SqlVS.exe EXOPLANETS.db "' + freshness_query + '"'
totalquery = totalquery.replace('\n',' ')

In [33]:
result = os.popen(totalquery).readlines()
result

['2020-02-08|8.0\n',
 '2020-03-30|4.0\n',
 '2020-05-14|8.0\n',
 '2020-06-17|5.0\n']

In [44]:
import pandas as pd
import numpy as np

# 两种接口
import plotly_express as px
import plotly.graph_objects as go

## 宜居程度信息

In [46]:
habitat_info_query = """
SELECT
  DATE_ADDED,
  AVG(HABITABILITY) AS AVG_HABITABILITY
FROM
  HABITABLES
GROUP BY
  DATE_ADDED;"""
habitat_info_query = '.\SqlVS.exe EXOPLANETS.db "' + habitat_info_query + '"'
habitat_info_query = habitat_info_query.replace('\n',' ')
result = os.popen(habitat_info_query).readlines()
x = []
y = []
for subresult in result:
    split_result = subresult.split('|')
    x.append(split_result[0])
    y.append(split_result[1].replace('\n',''))
df = pd.DataFrame({
    "date":x,
    "habitability":y
})
fig = px.bar(df,x="date",y="habitability")
fig.show()
# plt.bar(x,y)
# plt.show()

In [47]:
exoplanets_query = """
SELECT
  DATE_ADDED,
  COUNT(*) AS ROWS_ADDED
FROM
  EXOPLANETS
GROUP BY
  DATE_ADDED;"""
exoplanets_query = '.\SqlVS.exe EXOPLANETS.db "' + exoplanets_query + '"'
exoplanets_query = exoplanets_query.replace('\n',' ')
result = os.popen(exoplanets_query).readlines()
x = []
y = []
for subresult in result:
    split_result = subresult.split('|')
    x.append(split_result[0])
    y.append(split_result[1].replace('\n',''))
df = pd.DataFrame({
    "date_added":x,
    "rows_added":y
})
fig = px.bar(df,x="date_added",y="rows_added")
fig.show()

In [48]:
exoplanets_increase_query = """
WITH UPDATES AS(
  SELECT
    DATE_ADDED,
    COUNT(*) AS ROWS_ADDED
  FROM
    EXOPLANETS
  GROUP BY
    DATE_ADDED
)

SELECT
  DATE_ADDED,
  JULIANDAY(DATE_ADDED) - JULIANDAY(LAG(DATE_ADDED) OVER(
    ORDER BY DATE_ADDED
  )) AS DAYS_SINCE_LAST_UPDATE
FROM
  UPDATES;"""
exoplanets_increase_query = '.\SqlVS.exe EXOPLANETS.db "' + exoplanets_increase_query + '"'
exoplanets_increase_query = exoplanets_increase_query.replace('\n',' ')
result = os.popen(exoplanets_increase_query).readlines()
x = []
y = []
for subresult in result:
    split_result = subresult.split('|')
    x.append(split_result[0])
    y.append(split_result[1].replace('\n',''))
df = pd.DataFrame({
    "date_added":x,
    "days_since_last_update":y
})
fig = px.bar(df,x="date_added",y="days_since_last_update")
fig.show()

In [51]:
exoplanets_old_query = """

WITH UPDATES AS(
  SELECT
    DATE_ADDED,
    COUNT(*) AS ROWS_ADDED
  FROM
    EXOPLANETS
  GROUP BY
    DATE_ADDED
),

NUM_DAYS_UPDATES AS (
  SELECT
    DATE_ADDED,
    JULIANDAY(DATE_ADDED) - JULIANDAY(LAG(DATE_ADDED)
      OVER(
        ORDER BY DATE_ADDED
      )
    ) AS DAYS_SINCE_LAST_UPDATE
  FROM
    UPDATES
)

SELECT
  *
FROM
  NUM_DAYS_UPDATES
WHERE
  DAYS_SINCE_LAST_UPDATE > 1;"""
exoplanets_old_query = '.\SqlVS.exe EXOPLANETS.db "' + exoplanets_old_query + '"'
exoplanets_old_query = exoplanets_old_query.replace('\n',' ')
result = os.popen(exoplanets_old_query).readlines()
x = []
y = []
for subresult in result:
    split_result = subresult.split('|')
    x.append(split_result[0])
    y.append(split_result[1].replace('\n',''))
df = pd.DataFrame({
    "date_added":x,
    "days_since_last_update":y
})
df

,date_added,days_since_last_update
0,2020-02-08,8.0
1,2020-03-30,4.0
2,2020-05-14,8.0
3,2020-06-07,3.0
4,2020-06-17,5.0
5,2020-06-30,3.0


## Distribution

In [52]:
from plotly.subplots import make_subplots

In [58]:
exoplanets_nullrate_query = """
SELECT
  DATE_ADDED,
  CAST(
    SUM(
      CASE
        WHEN DISTANCE IS NULL THEN 1
        ELSE 0
      END
    ) AS FLOAT) / COUNT(*) AS DISTANCE_NULL_RATE,
  CAST(
    SUM(
      CASE
        WHEN G IS NULL THEN 1
        ELSE 0
      END
    ) AS FLOAT) / COUNT(*) AS G_NULL_RATE,
  CAST(
    SUM(
      CASE
        WHEN ORBITAL_PERIOD IS NULL THEN 1
        ELSE 0
      END
    ) AS FLOAT) / COUNT(*) AS ORBITAL_PERIOD_NULL_RATE,
  CAST(
    SUM(
      CASE
        WHEN AVG_TEMP IS NULL THEN 1
        ELSE 0
      END
    ) AS FLOAT) / COUNT(*) AS AVG_TEMP_NULL_RATE
FROM
  EXOPLANETS
GROUP BY
  DATE_ADDED;"""
exoplanets_nullrate_query = '.\SqlVS.exe EXOPLANETS.db "' + exoplanets_nullrate_query + '"'
exoplanets_nullrate_query = exoplanets_nullrate_query.replace('\n',' ')
result = os.popen(exoplanets_nullrate_query).readlines()
date = []
distance = []
g = []
orbital = []
avg = []
for subresult in result:
    split_result = subresult.split('|')
    date.append(split_result[0])
    distance.append(split_result[1])
    g.append(split_result[2])
    orbital.append(split_result[3])
    avg.append(split_result[4].replace('\n',''))
df0 = pd.DataFrame({
    "date_added":date,
    "distance_null_rate":distance
})
df1 = pd.DataFrame({
    "date_added":date,
    "g_null_rate":g
})
df2 = pd.DataFrame({
    "date_added":date,
    "orbital_null_rate":orbital
})
df3 = pd.DataFrame({
    "date_added":date,
    "avg_null_rate":avg
})
fig = px.bar(df3,x="date_added",y="avg_null_rate")
fig.show()

## Schema

In [60]:
exoplanets_columns_query = """
WITH CHANGES AS(
  SELECT
    DATE,
    COLUMNS AS NEW_COLUMNS,
    LAG(COLUMNS) OVER(ORDER BY DATE) AS PAST_COLUMNS
  FROM
    EXOPLANETS_COLUMNS
)

SELECT
    *
FROM
    CHANGES
WHERE
    NEW_COLUMNS != PAST_COLUMNS
ORDER BY
    DATE ASC;"""
exoplanets_columns_query = '.\SqlVS.exe EXOPLANETS.db "' + exoplanets_columns_query + '"'
exoplanets_columns_query = exoplanets_columns_query.replace('\n',' ')
result = os.popen(exoplanets_columns_query).readlines()
result

['2020-07-19|[\n',
 "    (0, '_id', 'TEXT', 0, None, 0),\n",
 "    (1, 'distance', 'REAL', 0, None, 0),\n",
 "    (2, 'g', 'REAL', 0, None, 0),\n",
 "    (3, 'orbital_period', 'REAL', 0, None, 0),\n",
 "    (4, 'avg_temp', 'REAL', 0, None, 0),\n",
 "    (5, 'date_added', 'TEXT', 0, None, 0),\n",
 "    (6, 'eccentricity', 'REAL', 0, None, 0),\n",
 "    (7, 'atmosphere', 'TEXT', 0, None, 0)\n",
 '  ]|[\n',
 "    (0, '_id', 'TEXT', 0, None, 0),\n",
 "    (1, 'distance', 'REAL', 0, None, 0),\n",
 "    (2, 'g', 'REAL', 0, None, 0),\n",
 "    (3, 'orbital_period', 'REAL', 0, None, 0),\n",
 "    (4, 'avg_temp', 'REAL', 0, None, 0),\n",
 "    (5, 'date_added', 'TEXT', 0, None, 0)\n",
 '  ]\n']